testing jupyter notebook from vscode

In [2]:
# imports
import pandas as pd
import requests
import altair as alt

In [3]:
alt.renderers.enable('altair_viewer')

RendererRegistry.enable('altair_viewer')

In [4]:
# set up alphavantage api stuff
# symbol = input()
api_key = 'LAJBI8EN52GC18OY'
base_url = 'https://www.alphavantage.co/query?'
params = {'function': 'TIME_SERIES_DAILY_ADJUSTED',
          'symbol': 'GME',
          'apikey': api_key}

In [5]:
# get response from api and format it a bit and put it into a dict
# this helps with the mulit level nested json by 'stripping' metadata header info
response = requests.get(base_url, params=params)
response_dict = response.json()
_, header = response.json()

In [6]:
# create a pandas dataframe from response_dict
df = pd.DataFrame.from_dict(response_dict[header], orient='index')
# strip the cols names down to bare bones header
df_cols = [i.split(' ')[1] for i in df.columns]
df.columns = df_cols

In [7]:
df['date'] = df.index

In [8]:
df

,open,high,low,close,adjusted,volume,dividend,split,date
2021-03-10,269.43,348.5,172.0,265.0,265.0,70716411,0.0000,1.0,2021-03-10
2021-03-09,217.71,249.85,208.5101,246.9,246.9,39099331,0.0000,1.0,2021-03-09
2021-03-08,154.89,210.8699,146.1,194.5,194.5,63565620,0.0000,1.0,2021-03-08
2021-03-05,128.17,151.53,127.5,137.74,137.74,30733673,0.0000,1.0,2021-03-05
2021-03-04,125.0,147.8699,115.3,132.35,132.35,32606893,0.0000,1.0,2021-03-04
...,...,...,...,...,...,...,...,...,...
2020-10-21,13.9,14.42,13.8,14.1,14.1,5361928,0.0000,1.0,2020-10-21
2020-10-20,14.03,14.143,13.665,13.86,13.86,6603951,0.0000,1.0,2020-10-20
2020-10-19,13.44,14.5,13.38,13.91,13.91,13169073,0.0000,1.0,2020-10-19
2020-10-16,13.77,13.9,13.08,13.31,13.31,11651578,0.0000,1.0,2020-10-16


In [9]:
#this saves a new df with a new index 01234 etc not the date string.
df1 = df.reset_index(drop=True)

In [10]:
#this works now with all cols as 'objects'. no need to convert 'date' col to datetime in pandas
chart = alt.Chart(df1).mark_circle(size=50).encode(
    x='date:T',
    y='open:Q',
    
    tooltip=['open','close','high','low']
).interactive()
chart.show()

Displaying chart at http://localhost:19311/


In [13]:
# notice the new index field and 'date' field
df1

,open,high,low,close,adjusted,volume,dividend,split,date
0,269.43,348.5,172.0,265.0,265.0,70716411,0.0000,1.0,2021-03-10
1,217.71,249.85,208.5101,246.9,246.9,39099331,0.0000,1.0,2021-03-09
2,154.89,210.8699,146.1,194.5,194.5,63565620,0.0000,1.0,2021-03-08
3,128.17,151.53,127.5,137.74,137.74,30733673,0.0000,1.0,2021-03-05
4,125.0,147.8699,115.3,132.35,132.35,32606893,0.0000,1.0,2021-03-04
...,...,...,...,...,...,...,...,...,...
95,13.9,14.42,13.8,14.1,14.1,5361928,0.0000,1.0,2020-10-21
96,14.03,14.143,13.665,13.86,13.86,6603951,0.0000,1.0,2020-10-20
97,13.44,14.5,13.38,13.91,13.91,13169073,0.0000,1.0,2020-10-19
98,13.77,13.9,13.08,13.31,13.31,11651578,0.0000,1.0,2020-10-16


In [15]:
df_difs = df1[['open','close']]

In [16]:
df_difs

,open,close
0,269.43,265.0
1,217.71,246.9
2,154.89,194.5
3,128.17,137.74
4,125.0,132.35
...,...,...
95,13.9,14.1
96,14.03,13.86
97,13.44,13.91
98,13.77,13.31


In [18]:
# this is computing the difference between 'open' and 'close' and putting result in the 'close' col after converting from object to numeric(float64). # otherwise math won't work
df_difs = df_difs[['open', 'close']].apply(pd.to_numeric).diff(axis=1)

In [28]:
df_diff = df_difs['close']

In [29]:
df_date = df1['date']

In [30]:
df_date

0     2021-03-10
1     2021-03-09
2     2021-03-08
3     2021-03-05
4     2021-03-04
         ...    
95    2020-10-21
96    2020-10-20
97    2020-10-19
98    2020-10-16
99    2020-10-15
Name: date, Length: 100, dtype: object

In [31]:
df_diff

0     -4.43
1     29.19
2     39.61
3      9.57
4      7.35
      ...  
95     0.20
96    -0.17
97     0.47
98    -0.46
99     1.84
Name: close, Length: 100, dtype: float64

In [38]:
df_posneg = pd.DataFrame(dict(date = df_date, diff = df_diff))

In [39]:
df_posneg

,date,diff
0,2021-03-10,-4.43
1,2021-03-09,29.19
2,2021-03-08,39.61
3,2021-03-05,9.57
4,2021-03-04,7.35
...,...,...
95,2020-10-21,0.20
96,2020-10-20,-0.17
97,2020-10-19,0.47
98,2020-10-16,-0.46


In [54]:


chart2 = alt.Chart(df_posneg).mark_bar().encode(
    x="date",
    y="diff:Q",
    tooltip=['diff','date'],
    color=alt.condition(
        alt.datum.diff > 0,
        alt.value("steelblue"),  # The positive color
        alt.value("orange")  # The negative color
    )
).properties(width=1200).interactive()
chart2.show()

Displaying chart at http://localhost:19311/
